In [3]:
import torch
import torchvision

dummy_input = torch.randn(10, 3, 224, 224, device='cpu')
model = torchvision.models.alexnet(pretrained=True)

# Providing input and output names sets the display names for values
# within the model's graph. Setting these does not change the semantics
# of the graph; it is only for readability.
#
# The inputs to the network consist of the flat list of inputs (i.e.
# the values you would pass to the forward() method) followed by the
# flat list of parameters. You can partially specify names, i.e. provide
# a list here shorter than the number of inputs to the model, and we will
# only set that subset of names, starting from the beginning.
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, input_names=input_names, output_names=output_names)


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth
100.0%


graph(%actual_input_1 : Float(10, 3, 224, 224),
      %learned_0 : Float(64, 3, 11, 11),
      %learned_1 : Float(64),
      %learned_2 : Float(192, 64, 5, 5),
      %learned_3 : Float(192),
      %learned_4 : Float(384, 192, 3, 3),
      %learned_5 : Float(384),
      %learned_6 : Float(256, 384, 3, 3),
      %learned_7 : Float(256),
      %learned_8 : Float(256, 256, 3, 3),
      %learned_9 : Float(256),
      %learned_10 : Float(4096, 9216),
      %learned_11 : Float(4096),
      %learned_12 : Float(4096, 4096),
      %learned_13 : Float(4096),
      %learned_14 : Float(1000, 4096),
      %learned_15 : Float(1000)):
  %17 : Float(10, 64, 55, 55) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[11, 11], pads=[2, 2, 2, 2], strides=[4, 4]](%actual_input_1, %learned_0, %learned_1) # /usr/local/lib/python3.6/dist-packages/torch/nn/modules/conv.py:350:0
  %18 : Float(10, 64, 55, 55) = onnx::Relu(%17) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1061:0
  %19 : Float

In [5]:
import onnx

# Load the ONNX model
model = onnx.load("alexnet.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %actual_input_1[FLOAT, 10x3x224x224]\n) initializers (\n  %learned_0[FLOAT, 64x3x11x11]\n  %learned_1[FLOAT, 64]\n  %learned_10[FLOAT, 4096x9216]\n  %learned_11[FLOAT, 4096]\n  %learned_12[FLOAT, 4096x4096]\n  %learned_13[FLOAT, 4096]\n  %learned_14[FLOAT, 1000x4096]\n  %learned_15[FLOAT, 1000]\n  %learned_2[FLOAT, 192x64x5x5]\n  %learned_3[FLOAT, 192]\n  %learned_4[FLOAT, 384x192x3x3]\n  %learned_5[FLOAT, 384]\n  %learned_6[FLOAT, 256x384x3x3]\n  %learned_7[FLOAT, 256]\n  %learned_8[FLOAT, 256x256x3x3]\n  %learned_9[FLOAT, 256]\n) {\n  %17 = Conv[dilations = [1, 1], group = 1, kernel_shape = [11, 11], pads = [2, 2, 2, 2], strides = [4, 4]](%actual_input_1, %learned_0, %learned_1)\n  %18 = Relu(%17)\n  %19 = MaxPool[kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [2, 2]](%18)\n  %20 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%19, %learned_2, %learned_3)\n  %21 = Relu(%20)\n  %22 = MaxPool[kernel

In [7]:
from onnx_tf.backend import prepare

tf_rep = prepare(model)

AttributeError: module 'tensorflow_addons' has no attribute 'utils'